In [ ]:
!pip install gower

In [ ]:
import pandas as pd
import gower
from scipy.stats import rankdata
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/조달청 공모전/제목없는 폴더/final.csv')
df.drop(axis=1, columns='Unnamed: 0',inplace=True)
df.drop(axis=1,columns=['낙찰자결정방법','입찰계약방법'],inplace=True)
df_raw = pd.read_csv('/content/drive/Shareddrives/조달청 공모전/제목없는 폴더/0131/2019공사공고.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,7,9,18,34,35,37,47,48,52,55,58,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
target_gongo = df.iloc[70000] #test를 위한 sample 공고선택
target_gongo_df = pd.DataFrame(columns= df.columns.tolist())
target_gongo_df.loc[0] = target_gongo

In [ ]:
def gower_test(df,target_gongo_df,df_raw):
  sampling_df = df[:10000] #데이터가 너무 많아 개수 지정
  df_final = pd.concat ( [sampling_df, target_gongo_df]) #타겟이 dataFrame 형태 (한줄)로 들어 온다고 가정
  df_id = sampling_df['입찰공고번호'] #공고번호는 살려놓음(추후 사용)
  df_id.loc[-1]= target_gongo_df.입찰공고번호.tolist()[0]
  df_final.drop(axis=1,columns='입찰공고번호',inplace=True) #유사도 예측시 입찰공고번호는 제외, 나중에 다시 add
  distance_matrix = gower.gower_matrix(df_final) #gower 라이브러리를 통해 유사도 matrix 도출
  array = distance_matrix[-1] #맨 마지막 df에 target을 붙인 형태이므로 마지막 row 선택
  ranks = rankdata(array)
  df_final['rank'] = ranks.tolist() #target과 가장 가까운 rank 산출(유사도 계수가 낮은 행 선택)
  df_final['입찰공고번호'] = df_id.tolist()
  df_rank_index_df = df_final[(df_final['rank']) < 5].sort_values('rank') # 1,2,3위까지 도출 (가장 가까운 것은 자신 row)
  df_rank_list = df_rank_index_df['입찰공고번호'].tolist()
  return df_raw[df_raw['입찰공고번호'].isin(df_rank_list)] #원래 공고번호까지 붙여줘서 raw_data에서 출력

In [ ]:
gower_test(df,target_gongo_df,df_raw)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


,등록유형,조달구분,공고시스템명,입찰공고번호,입찰공고차수,공고게시일자,공고명,공사현장,입찰방식,낙찰자결정적용법규,낙찰자결정방법,입찰계약방법,예가방법,긴급공고여부,업종제한여부,업종제한내용,지역제한내용,지역의무공동도급여부,공동도급지역,공동도급구성방식명,기초금액,관급자설치관급자재금액,도급자설치관급자재금액,추정금액,추정가격,예정가격,입찰서개시일자,입찰서마감일자,개찰일자,공고기관명,공고기관코드,실수요기관명,수요기관명,수요기관코드,수요기관구분,수요기관지역명,입찰진행구분,입찰진행내용,낙찰자결정여부,참가수,개찰1순위업체목록내용,개찰1순위투찰율,개찰1순위투찰금액,낙찰업체목록내용,낙찰업체투찰률,낙찰업체투찰금액,낙찰하한율,적격심사평가기준,적격심사주공종,적격심사주공종추정금액,적격심사주공종추정가격,적격심사업종평가비율,PQ심사접수방식,PQ심사신청서접수기한,국제입찰여부,국제입찰사유,정정공고여부,취소공고여부,발주계획통합번호,조달요청번호,공공조달분류명,사전규격등록번호,참조번호,이전입찰공고번호
4010,나라장터(G2B),자체조달,NaN,20190238327,0,20190228,행정리(행정1지) 정비공사,경상북도 칠곡군,전자입찰,지방계약법,수의(견적제출),수의(소액),복수예가,N,Y,[보링.그라우팅공사업(0019)],경상북도 칠곡군[47850],N,NaN,NaN,67700000.0,0.0,2300000.0,70000000.0,61545455.0,66773075.0,20190304.0,20190307.0,20190307,경상북도 칠곡군,5220000,경상북도 칠곡군,경상북도 칠곡군,5220000,지자체,경상북도 칠곡군,개찰완료,NaN,Y,4.0,뉴지오텍(주)[6468600488],88.827,59312600.0,뉴지오텍(주)[6468600488],88.827,59312600.0,87.745,NaN,NaN,NaN,NaN,NaN,없음,NaN,N,NaN,N,N,3-1-2019-5220000-000033,NaN,보링그라우팅공사,NaN,칠곡군 공고 제2019-68호,NaN
4198,나라장터(G2B),자체조달,NaN,20190301556,0,20190304,주곡 마을하수처리장 개체공사(건축),경상북도 영양군,전자입찰,지방계약법,수의(견적제출),수의(소액),복수예가,N,Y,[건축공사업(0002)]업종 또는[토목건축공사업(0003)],경상북도 영양군[47760],N,NaN,NaN,59800000.0,0.0,0.0,59800000.0,54363637.0,59389550.0,20190305.0,20190308.0,20190308,경상북도 영양군,5170000,경상북도 영양군,경상북도 영양군,5170000,지자체,경상북도 영양군,개찰완료,NaN,Y,2.0,엔에스건설합자회사[5028142183],88.932,52816700.0,엔에스건설합자회사[5028142183],88.932,52816700.0,87.745,NaN,NaN,NaN,NaN,NaN,없음,NaN,N,NaN,N,N,3-1-2019-5170000-000018,NaN,건축공사,NaN,영양군 공고 제2019-160호,NaN
18562,나라장터(G2B),자체조달,NaN,20190205769,0,20190210,월항도로(군도16호선) 차선도색공사,경상북도 성주군,전자입찰,지방계약법,제한최저,수의(소액),복수예가,N,Y,[도장공사업(0010)],경상북도 성주군[47840],N,NaN,NaN,70800000.0,0.0,0.0,70800000.0,64363637.0,70877225.0,20190211.0,20190214.0,20190214,경상북도 성주군,5210000,경상북도 성주군,경상북도 성주군,5210000,지자체,경상북도 성주군,개찰완료,NaN,Y,5.0,주식회사 영창안전[5108122140],88.901,63011200.0,주식회사 영창안전[5108122140],88.901,63011200.0,87.745,NaN,NaN,NaN,NaN,NaN,없음,NaN,N,NaN,N,N,3-1-2019-5210000-001715,NaN,도장공사,NaN,성주군 공고 제2019-106호,NaN
117907,나라장터(G2B),자체조달,NaN,20191212649,0,20191209,풍양 생태공원 생태탐방로 보수공사,경상북도 예천군,전자입찰,지방계약법,수의(견적제출),수의(소액)-견적입찰(2인 이상 견적 제출),복수예가,N,Y,[조경시설물설치공사업(0049)],경상북도 예천군[47900],N,NaN,NaN,38900000.0,0.0,47100000.0,86000000.0,35363636.0,38575800.0,20191209.0,20191212.0,20191212,경상북도 예천군,5230000,경상북도 예천군,경상북도 예천군,5230000,지자체,경상북도 예천군,개찰완료,NaN,Y,3.0,주식회사 송학조경[5028618368],88.850,34274890.0,주식회사 송학조경[5028618368],88.850,34274890.0,87.745,NaN,NaN,NaN,NaN,NaN,없음,NaN,N,NaN,N,N,3-1-2019-5230000-000382,NaN,조경시설물설치공사,NaN,예천군 공고 제2019-1350호,NaN
